In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
result = pd.DataFrame(test.Applicant_ID)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.isnull().sum()

In [ ]:
train=train.fillna(-999)
train

In [ ]:
train.set_index(['Applicant_ID'],inplace=True)

In [ ]:
train

In [ ]:
train['default_status']=train.default_status.apply(lambda x:1 if x=='yes' else 0)
train['form_field47']=train.form_field47.apply(lambda y:0 if y=='charge' else 1)

In [ ]:
train

In [ ]:
train.corr()

In [ ]:
#Prepare the Data set for training the model
target = train['default_status']
train = train.drop(['default_status'], axis=1)

In [ ]:
scaler = MinMaxScaler() 
train = scaler.fit_transform(train)
train

In [ ]:
# PREPROCESS THE TEST DATA
test['form_field47']=test.form_field47.apply(lambda y:0 if y=='charge' else 1)
test.set_index(['Applicant_ID'],inplace=True)
test=test.fillna(-999)
test

In [ ]:
test = scaler.fit_transform(test)
test

In [ ]:
rfc= RandomForestClassifier()
Catboost= CatBoostClassifier(verbose=False)
LGBM= LGBMClassifier()
Classifiers=[('rfc',rfc),('Catboost',Catboost),('LGBM',LGBM)]
vc=VotingClassifier(estimators=Classifiers,voting='soft')

In [ ]:
# StraifiedKFold
splits = 25
skfold = StratifiedKFold(n_splits=splits, random_state=42, shuffle=True)
scores = []
preds = []
i = 1

for train_split, test_split in skfold.split(train,target):
    rf = vc
    x_train, x_test, y_train, y_test = train[train_split], train[test_split], target[train_split], target[test_split]
    rf.fit(x_train, y_train)
    score = roc_auc_score(y_test, rf.predict_proba(x_test)[:,1])   
    pred = rf.predict_proba(test)
    scores.append(score)
    preds.append(pred)
    print('Roc score {} split: '.format(i), score)
    i +=1
 
test_pred = np.mean(preds, axis=0)
print('Average Roc Score: ', np.mean(scores))

In [ ]:
pd.DataFrame({"Applicant_ID": result, "Default_Status": test_pred}).to_csv("rf_SKfold_5splits-submission.csv", index = False)